In [68]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [61]:
import pandas as pd
import spacy
from sklearn.linear_model import SGDClassifier, LogisticRegression

nlp = spacy.load("en")

def get_word_vectors(docs):
    return [nlp(doc).vector for doc in docs]

train = pd.read_csv('ds4-which-whisk/train.csv')
test = pd.read_csv('ds4-which-whisk/test.csv')


In [62]:
train1 = train[~train['category'].isna()]

In [5]:
test.head()

,id,author,description,price,ratingValue,pert_alcohol
0,955,Fred Minnick,"Think carnival aromas—the good ones, anyway—me...",36.0,90,50.0
1,3532,Lew Bryson,"A blend of three bourbons, between 6 and 12 ye...",90.0,82,49.3
2,1390,Davin de Kergommeaux,"The nose is focused on cereal, hints of fresh ...",48.0,89,45.0
3,1024,Gavin Smith,Swiss-based Chapter 7 released this 19 year ol...,180.0,90,55.8
4,1902,Gavin Smith,Valkyrie replaces the current Dark Origins exp...,71.0,87,45.9


In [59]:
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score,confusion_matrix

In [60]:
STOP_WORDS = nlp.Defaults.stop_words.union(['i', 'and', ' ', 'to', 'be', '\'s', 'with', 'the', 'and', 'i'])

In [47]:
train1.shape, test.shape

((2586, 7), (288, 6))

In [48]:
test.tail()

,id,author,description,price,ratingValue,pert_alcohol
283,1834,John Hansell,"Aged in a sherry cask, and the sherry is clean...",80.0,87,55.5
284,2848,Dave Broom,There have been some legendary Hart Brothers b...,78.0,84,55.5
285,3874,Jeffery Lindenmuth,"Aromas hint at a bitterly astringent, almost b...",85.0,79,50.0
286,2377,Fred Minnick,"Think corn: roasted, canned, creamed, mashed, ...",75.0,86,63.7
287,2243,John Hansell,Amber gold color. Medium bodied and slightly o...,35.0,86,45.0


In [72]:
tf_idf=TfidfVectorizer(ngram_range=(1,1),stop_words=STOP_WORDS,analyzer='word',token_pattern=r'\w{1,}')
review_tfidf=tf_idf.fit_transform(train1['description'])
review_tfidf.get_shape()

(2586, 9120)

In [73]:
x_train_tf,x_test_tf,y_train_tf,y_test_tf=train_test_split(review_tfidf,train1['category'],train_size=0.8,random_state=100)

In [74]:
logit=LogisticRegression(class_weight='balanced',multi_class='multinomial',solver='lbfgs')
logit.fit(x_train_tf,y_train_tf)
logit.get_params()

{'C': 1.0,
 'class_weight': 'balanced',
 'dual': False,
 'fit_intercept': True,
 'intercept_scaling': 1,
 'max_iter': 100,
 'multi_class': 'multinomial',
 'n_jobs': None,
 'penalty': 'l2',
 'random_state': None,
 'solver': 'lbfgs',
 'tol': 0.0001,
 'verbose': 0,
 'warm_start': False}

In [75]:
### Test over the data:
predictions=logit.predict(x_test_tf)

In [77]:
submission = pd.DataFrame({'id':test['id'],'category':pred})
submission['category'] = submission['category'].astype(int)
submission.head()

ValueError: array length 518 does not match index length 288

In [13]:
filename = 'whiskey_preds02.csv'

submission.to_csv(filename,index=False)

print('Saved file: ' + filename)

Saved file: whiskey_preds.csv
